# Цели:

1.Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?
2.Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?

In [1]:
import pandas as pd
from datetime import datetime, timedelta
#https://drive.google.com/drive/folders/1LnZQNmVtp2ui29W5xgOIe-0cszMz88Bz?usp=sharing датасет
events_df=pd.read_csv('./data/Events.csv')
#Создание списка пользователей, зарегестировавшихся в 2018 году
cond=(events_df.start_time>='2018-01-01') & (events_df.start_time<'2019-01-01') & (events_df.event_type=='registration')
registered=events_df[cond].user_id.to_list()
events=events_df[events_df.user_id.isin(registered)] #сортировка по дате
#Преобразование формата даты
events.start_time=pd.to_datetime(events.start_time, format='%Y-%m-%dT%H:%M:%S')

/home/fktr0/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [2]:
#https://drive.google.com/drive/folders/1LnZQNmVtp2ui29W5xgOIe-0cszMz88Bz?usp=sharing датасет
purchase_df=pd.read_csv('./data/purchase.csv')
purchase_df['event_type']='purchase' # создание столбца event_type со значением purchase
purchase=purchase_df[purchase_df.user_id.isin(registered)] #сортировка по дате
#Преобразование формата даты
purchase.event_datetime=pd.to_datetime(purchase.event_datetime, format='%Y-%m-%dT%H:%M:%S')

# Определение промежутков времени между регистрацией и выборов одного из уровней сложности

In [3]:
#Создадим датафрейм, в котором будут храниться только собития с event_type=registration и проверим, чтобы на одного пользователя
#была одна регестрация
registration_df=events[events['event_type']=='registration']
print(registration_df['user_id'].value_counts().mean())
#Оставим в датафрейме только те данные, которые нам нужны для анализа
registration_df=registration_df[['user_id','start_time']].rename(columns={'start_time':'registration_time'})

1.0


In [4]:
#Создание датафрейма, в котором содержаться данные о пользователях, которые выбрали уровень сложности и проверим, чтобы на 
#одного пользователя было одно событие
level_choice_df=events[events['event_type']=='level_choice']
print(level_choice_df['user_id'].value_counts().mean())


1.0


In [5]:
#Объединение датафреймов level_choice_df и purchase
level_choice_df=level_choice_df.rename(columns={'id':'level_choice_id'})
purchase=purchase.rename(columns={'id':'purchase_id'})
total_events=pd.concat([events,purchase],sort=False)
total_events= total_events.reset_index(drop=True).sort_values('start_time')


In [6]:
#Создадим датафреймы с выбором каждого уровня сложности и создадим массив из уникальных user_id
level_choice_easy_df=level_choice_df[level_choice_df['selected_level']=='easy']['user_id'].unique()
level_choice_medium_df=level_choice_df[level_choice_df['selected_level']=='medium']['user_id'].unique()
level_choice_hard_df=level_choice_df[level_choice_df['selected_level']=='hard']['user_id'].unique()
#Отфильтруем датафрейм purchase
purchase_easy_df=purchase[purchase['user_id'].isin(level_choice_easy_df)]
purchase_medium_df=purchase[purchase['user_id'].isin(level_choice_medium_df)]
purchase_hard_df=purchase[purchase['user_id'].isin(level_choice_hard_df)]
#Оставим в датафреймах только те данные, которые нужны для анализа
purchase_easy_df=purchase_easy_df[['user_id','event_datetime','amount']].rename(columns={'event_datetime':'easy_choice_time'})
purchase_medium_df=purchase_medium_df[['user_id','event_datetime','amount']].rename(columns={'event_datetime':'medium_choice_time'})
purchase_hard_df=purchase_hard_df[['user_id','event_datetime','amount']].rename(columns={'event_datetime':'hard_choice_time'})

### Определим время между регистрацией и оплатой пользователей, которые выбрали уровень easy

In [7]:
merged_easy_df=registration_df.merge(purchase_easy_df,on='user_id',how='inner')
merged_easy_df['timedelta']=merged_easy_df['easy_choice_time']-merged_easy_df['registration_time']
print(merged_easy_df['timedelta'].describe())

count                          189
mean     3 days 22:10:23.211640211
std      2 days 07:14:41.062010764
min                0 days 04:36:58
25%                2 days 01:12:12
50%                3 days 11:00:23
75%                5 days 10:24:59
max               11 days 00:35:04
Name: timedelta, dtype: object


### Определим время между регистрацией и оплатой пользователей, которые выбрали уровень medium

In [8]:
merged_medium_df=registration_df.merge(purchase_medium_df,on='user_id',how='inner')
merged_medium_df['timedelta']=merged_medium_df['medium_choice_time']-merged_medium_df['registration_time']
print(merged_medium_df['timedelta'].mean())
print(merged_medium_df['timedelta'].describe())

4 days 06:12:06.576883384
count                          969
mean     4 days 06:12:06.576883384
std      2 days 06:25:57.480868026
min                0 days 08:39:24
25%                2 days 08:46:51
50%                4 days 03:35:26
75%                5 days 23:51:27
max               10 days 20:34:02
Name: timedelta, dtype: object


### Определим время между регистрацией и оплатой пользователей, которые выбрали уровень hard

In [9]:
merged_hard_df=registration_df.merge(purchase_hard_df,on='user_id',how='inner')
merged_hard_df['timedelta']=merged_hard_df['hard_choice_time']-merged_hard_df['registration_time']
print(merged_hard_df['timedelta'].describe())

count                          442
mean     3 days 14:55:19.257918552
std      1 days 22:22:52.441896774
min                0 days 09:41:39
25%         1 days 23:36:25.500000
50%         3 days 10:10:04.500000
75%         5 days 03:30:07.750000
max                8 days 14:21:29
Name: timedelta, dtype: object


# Определение зависимости оплаты от выбранного уровня сложности

### Процент совершивших оплату от числа пользователей, которые выбрали уровень easy

In [10]:
persent_easy=len(purchase_easy_df)/len(level_choice_easy_df)
print('Процент совершивших оплату от числа пользователей, которые выбрали уровень easy: {:.2%}'.format(persent_easy))
print('Средний размер платежа: {}'.format(purchase_easy_df['amount'].mean()))

Процент совершивших оплату от числа пользователей, которые выбрали уровень easy: 7.72%
Средний размер платежа: 114.94708994708995


### Процент совершивших оплату от числа пользователей, которые выбрали уровень medium

In [11]:
persent_medium=len(purchase_medium_df)/len(level_choice_medium_df)
print('Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: {:.2%}'.format(persent_medium))
print('Средний размер платежа: {}'.format(purchase_medium_df['amount'].mean()))

Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: 20.86%
Средний размер платежа: 109.52012383900929


### Процент совершивших оплату от числа пользователей, которые выбрали уровень hard

In [12]:
persent_hard=len(purchase_hard_df)/len(level_choice_hard_df)
print('Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: {:.2%}'.format(persent_hard))
print('Средний размер платежа: {}'.format(purchase_hard_df['amount'].mean()))

Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: 35.39%
Средний размер платежа: 111.59502262443439


# Выводы
## 1.Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?
### Уровень easy:
Среднее время: 3 дня 22:10:23  
Медианное время: 3 дня 11:00:23  
### Уровень medium:
Среднее время: 4 дня 06:12:06  
Медианное время: 4 дня 03:35:26  
### Уровень hard:
Среднее время: 3 дня 14:55:19  
Медианное время: 3 дня 10:10:04  
  
У пользователей, которые выбрали разных уровень сложности, нет значительной разницы во времени от момента регистрации до момента оплаты. Однако, быстрее всего оплачивают пакет вопросов пользователи, которые выбрали уровень hard, а медленнее всего плавичают пользователи, которые выбрали уровень medium.
  
## 2.Зависит ли вероятность оплаты от выбранного пользователем уровня сложности?
### Уровень easy:
Процент совершивших оплату от числа пользователей, которые выбрали уровень easy: 7.72%
### Уровень medium:
Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: 20.86%
### Уровень hard:
Процент совершивших оплату от числа пользователей, которые выбрали уровень medium: 35.39%  
  
Есть прямая зависимость между выбором пользоваетелем уровня сложности и вероятности покупки пакета вопросов. Чем более высокий уровень сложности выбирает пользователь, тем выше вероятность покупки им пакета вопросов.